In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q chromadb pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:0

In [3]:
from typing import List, Dict
from sentence_transformers import SentenceTransformer

import json
import chromadb

# 1. Embedding QUY CHẾ TỔ CHỨC VÀ QUẢN LÝ ĐÀO TẠO TRÌNH ĐỘ ĐẠI HỌC

In [4]:
# ========= 1. Load chunks từ file JSON =========

def load_chunks(path: str) -> List[Dict]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # đảm bảo data là list
    if not isinstance(data, list):
        raise ValueError("File JSON phải chứa một list các chunk ([]).")

    # filter những chunk thiếu text
    cleaned = []
    for i, c in enumerate(data):
        text = (c.get("text") or "").strip()
        if not text:
            # nếu muốn debug, có thể print cảnh báo
            # print(f"Warning: chunk index {i} không có text, bỏ qua.")
            continue

        # nếu thiếu chunk_id thì tạo tạm
        if "chunk_id" not in c:
            c["chunk_id"] = f"chunk_{i}"

        cleaned.append(c)

    print(f"Loaded {len(cleaned)} chunks (sau khi lọc những chunk trống).")
    return cleaned

In [5]:
# ========= 2. Chuẩn bị text_list, ids, metadata =========

def build_inputs(chunks: List[Dict]):
    texts = []
    ids = []
    metadatas = []

    for c in chunks:
        text = (c.get("text") or "").strip()
        if not text:
            continue

        chunk_id = c.get("chunk_id")
        if not chunk_id:
            continue

        texts.append(text)
        ids.append(chunk_id)

        meta = {
            "doc_id": c.get("doc_id", ""),
            "chapter_title": c.get("chapter_title", ""),
            "article_title": c.get("article_title", ""),
            "chunk_index": c.get("chunk_index", 0),
            "chunk_type": c.get("chunk_type", ""),
        }
        metadatas.append(meta)

    print(f"Prepared {len(texts)} texts for embedding.")
    return texts, ids, metadatas

In [6]:
# ========= 3. Load model AITeamVN/Vietnamese_Embedding =========

def load_embedding_model(device: str = "cuda") -> SentenceTransformer:
    """
    device:
       - 'cuda'
    """
    model_name = "AITeamVN/Vietnamese_Embedding"
    print(f"Loading model: {model_name} on {device} ...")
    model = SentenceTransformer(model_name, device=device)
    # theo model card, max_seq_length = 2048, có thể set lại cho chắc
    model.max_seq_length = 2048  # :contentReference[oaicite:2]{index=2}
    return model

In [7]:
# ========= 4. Tạo/Persist vectorDB (Chroma) =========

def init_chroma_collection(persist_dir: str = "/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_qd1830",
                           collection_name: str = "qd_1830"):
    """
    Tạo Chroma collection, lưu local vào thư mục persist_dir.
    """
    client = chromadb.PersistentClient(path=persist_dir)

    # nếu collection đã tồn tại, get ra; nếu chưa thì tạo mới
    try:
        collection = client.get_collection(collection_name)
        print(f"Using existing collection: {collection_name}")
    except chromadb.errors.NotFoundError:
        collection = client.create_collection(name=collection_name)
        print(f"Created new collection: {collection_name}")

    return collection

In [8]:
# ========= 5. Embed & upsert into Chroma =========

def embed_and_index(chunks_path: str,
                    device: str = "cuda",
                    persist_dir: str = "./chroma_qd1830",
                    collection_name: str = "qd_1830",
                    batch_size: int = 64):
    # 5.1 Load chunks
    chunks = load_chunks(chunks_path)

    # 5.2 Build input lists
    texts, ids, metadatas = build_inputs(chunks)

    # 5.3 Load model
    model = load_embedding_model(device=device)

    # 5.4 Init Chroma
    collection = init_chroma_collection(
        persist_dir=persist_dir,
        collection_name=collection_name
    )

    # 5.5 Embed theo batch và upsert
    total = len(texts)
    print(f"Start embedding + indexing {total} chunks ...")

    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)
        batch_texts = texts[start:end]
        batch_ids = ids[start:end]
        batch_meta = metadatas[start:end]

        # tạo embedding
        # normalize_embeddings=True để tối ưu dot-product similarity :contentReference[oaicite:3]{index=3}
        batch_emb = model.encode(
            batch_texts,
            normalize_embeddings=True
        ).tolist()

        # upsert vào collection
        collection.upsert(
            ids=batch_ids,
            embeddings=batch_emb,
            metadatas=batch_meta,
            documents=batch_texts,
        )

        print(f"Indexed {end}/{total} chunks")

    print("Done embedding & indexing.")


In [9]:
# ========= 6. Hàm search thử để debug =========

def search_in_qd1830(query: str,
                     device: str = "cuda",
                     persist_dir: str = "/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_qd1830",
                     collection_name: str = "qd_1830",
                     top_k: int = 5):
    # load model
    model = load_embedding_model(device=device)

    # init collection
    client = chromadb.PersistentClient(path=persist_dir)
    collection = client.get_collection(collection_name)

    # embedding query
    q_emb = model.encode([query], normalize_embeddings=True).tolist()

    # query collection
    res = collection.query(
        query_embeddings=q_emb,
        n_results=top_k
    )

    # res là dict gồm 'ids', 'documents', 'metadatas', 'distances'
    for i in range(len(res["ids"][0])):
        print("=" * 80)
        print(f"Rank #{i+1}")
        print("ID:      ", res["ids"][0][i])
        print("Score:   ", res["distances"][0][i])
        meta = res["metadatas"][0][i]
        print("Doc ID:  ", meta.get("doc_id"))
        print("Chapter: ", meta.get("chapter_title"))
        print("Article: ", meta.get("article_title"))
        print("Chunk i: ", meta.get("chunk_index"))
        print("--- Text ---")
        print(res["documents"][0][i][:500], "...")
    print("=" * 80)

In [10]:
# 1) Embed & index
embed_and_index(
    chunks_path="/content/drive/MyDrive/NLP/RAG_final/Chunking/qd_1830_chunks_debug.json",
    device="cuda",        # nếu đang dùng GPU Colab: "cuda"
    persist_dir="/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_qd1830",
    collection_name="qd_1830",
    batch_size=64
)

Loaded 120 chunks (sau khi lọc những chunk trống).
Prepared 120 texts for embedding.
Loading model: AITeamVN/Vietnamese_Embedding on cuda ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Created new collection: qd_1830
Start embedding + indexing 120 chunks ...
Indexed 64/120 chunks
Indexed 120/120 chunks
Done embedding & indexing.


In [13]:
# 2) Thử search 1 câu hỏi
print("\n===== TEST SEARCH =====")

search_in_qd1830(
    query="Chương Trình đào tạo của Đại học Tôn Đức Thắng được xây dựng theo đơn vị gì ?",
    device="cuda",
    persist_dir="/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_qd1830",
    collection_name="qd_1830",
    top_k=5
)


===== TEST SEARCH =====
Loading model: AITeamVN/Vietnamese_Embedding on cuda ...
Rank #1
ID:       qd_1830_2021_art_0
Score:    1.064664363861084
Doc ID:   qd_1830_2021
Chapter:  Chương I. NHỮNG QUY ĐỊNH CHUNG
Article:  Điều 1. Phạm vi và đối tượng áp dụng
Chunk i:  0
--- Text ---
1. Quy chế này quy định tổ chức và quản lý đào tạo trình độ đại học hệ chính quy theo hệ thống tín chỉ tại Trường Đại học Tôn Đức Thắng, bao gồm: Chương trình đào tạo, tổ chức đào tạo, đánh giá kết quả học tập, xét và công nhận tốt nghiệp và những quy định khác. 2. Quy chế này áp dụng đối với sinh viên chương trình đào tạo tiêu chuẩn, chương trình đào tạo chất lượng cao, chương trình đào tạo giảng dạy bằng tiếng Anh hệ chính quy trình độ đại học tại Trường Đại học Tôn Đức Thắng (sau đây gọi tắt  ...
Rank #2
ID:       qd_1830_2021_art_1
Score:    1.1088676452636719
Doc ID:   qd_1830_2021
Chapter:  Chương I. NHỮNG QUY ĐỊNH CHUNG
Article:  Điều 2. Chương trình đào tạo
Chunk i:  0
--- Text ---
1. Chương trình đà